In [1]:
import sys; sys.path.append("../..") # Allow imports from parent directory

In [46]:
iterator = iter(niter_print)

In [44]:
for i in niter_print:
    print(i)
    if i>100:break

132


In [49]:
next(iterator)

2

In [45]:
import itertools
niter_print = itertools.chain(range(4),range(4,16,2),itertools.count(16,4))
#for i in niter_print:
#    print(i)
#    if i>=50:
#        break

In [10]:
l=[1,2,3,4]
iter(l)

AttributeError: 'list_iterator' object has no attribute 'next'

In [3]:
False is not None

True

In [1]:
2.*3.**-1

0.6666666666666666

In [2]:
import numpy as np
import scipy.sparse; import scipy.sparse.linalg

In [3]:
import NumericalSchemes.AutomaticDifferentiation as ad
import NumericalSchemes.FiniteDifferences as fd

In [4]:
x0=np.linspace(0,np.pi,4)
gridScale=x0[1]-x0[0]
u0=np.sin(x0)
v0=np.cos(x0)

## Without intermediate states

In [5]:
# Create an empty history
rev = ad.Reverse2.empty()
# Create AD variables w.r.t which the gradient will be required
u1 = rev.identity(constant=u0)
v1 = rev.identity(constant=v0)

result = (u1**2+v1**2).sum()

grad = rev.gradient(result)
hess = rev.hessian(result)

In [6]:
hess(grad)

[ 0.00000000e+00  1.73205081e+00  1.73205081e+00  2.44929360e-16
  2.00000000e+00  1.00000000e+00 -1.00000000e+00 -2.00000000e+00] [0 1 2 3 4 5 6 7]


array([ 0.00000000e+00,  3.46410162e+00,  3.46410162e+00,  4.89858720e-16,
        4.00000000e+00,  2.00000000e+00, -2.00000000e+00, -4.00000000e+00])

In [7]:
result

spAD2(array(4.), array([ 0.00000000e+00,  1.73205081e+00,  1.73205081e+00,  2.44929360e-16,
        2.00000000e+00,  1.00000000e+00, -1.00000000e+00, -2.00000000e+00]), array([0, 1, 2, 3, 4, 5, 6, 7]), array([2., 2., 2., 2., 2., 2., 2., 2.]), array([0, 1, 2, 3, 4, 5, 6, 7]), array([0, 1, 2, 3, 4, 5, 6, 7]))

## With intermediate states

In [13]:
def Scheme(u,h):
    return fd.DiffUpwind(u,(1,),h,padding=0.)
operator = Scheme(ad.Sparse.identity(u0.shape),gridScale)
matrix = scipy.sparse.coo_matrix(operator.triplets()).tocsr()

In [40]:
ad.reload_submodules()

In [44]:
# Create an empty history
rev = ad.Reverse2.empty()
# Create AD variables w.r.t which the gradient will be required
u1 = rev.identity(constant=u0)
u2 = rev.apply_linear_mapping(matrix,u1)
result = (u2**2).sum()

grad = rev.gradient(result)
hess = rev.hessian(result)

In [45]:
grad

array([-1.57944094,  1.57944094,  1.57944094, -1.57944094])

In [46]:
hess(grad)

[-1.57944094e+00  1.57944094e+00  1.57944094e+00 -1.57944094e+00
  3.01651000e+00  1.11022302e-15 -3.01651000e+00  1.50825500e+00] [4 5 6 7]


array([-5.76110972,  5.76110972,  5.76110972, -8.64166459])

In [51]:
# Control with Dense AD

u1 = ad.Dense2.identity(constant=u0)
u2 = ad.apply_linear_mapping(matrix,u1)
result = (u2**2).sum()

print("gradient",result.coef1)
print("hessian in direction of gradient",np.dot(result.coef2,result.coef1))

gradient [-1.57944094  1.57944094  1.57944094 -1.57944094]
hessian in direction of gradient [-5.76110972  5.76110972  5.76110972 -8.64166459]
